# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [2]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [4]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

ds = load_dataset(dataset_name)


In [6]:
# TODO: Print a record
print(ds.shape)
print()
print(ds)

{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [7]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [8]:
# TODO: Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
# TODO: Print number of trainable parameters
print(print_trainable_model_params(model))

Trainable parameters: 247577856
Total parameters: 247577856
Percentable of trainable parameters: 100.00%


In [20]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [11]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [12]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_ds = ds.map(tokenize_fn, batched=True)



Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# TODO: Verify prepared data
idx = 100
for k, v in tokenized_ds['train'][idx].items():
   print(f'key: {k}')
   print(f'value: {v}')
   print('----------')

In [16]:
print(tokenizer.decode(tokenized_ds['train'][idx]['input_ids'], skip_special_tokens=True))
print()
print(tokenizer.decode(tokenized_ds['train'][idx]['labels'], skip_special_tokens=True))

Summarize the following dialogue. #Person1#: I have a problem with my cable. #Person2#: What about it? #Person1#: My cable has been out for the past week or so. #Person2#: The cable is down right now. I am very sorry. #Person1#: When will it be working again? #Person2#: It should be back on in the next couple of days. #Person1#: Do I still have to pay for the cable? #Person2#: We're going to give you a credit while the cable is down. #Person1#: So, I don't have to pay for it? #Person2#: No, not until your cable comes back on. #Person1#: Okay, thanks for everything. #Person2#: You're welcome, and I apologize for the inconvenience. Summary:

#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.


In [17]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_ds = tokenized_ds.remove_columns([
   'id', 'dialogue', 'summary', 'topic'
])

In [18]:
# TODO: Verify dataset again
print(tokenized_ds)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [19]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

CUDA available:  False


## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [21]:
# TODO: Configure LoRA
# r - small = be better at what it is doing, not to learn anything new
# r - large = learn new concepts, higher abstraction
lora_config = LoraConfig(
   r=16,
   lora_alpha=16,
   target_modules=['q', 'v'],
   bias="none",
   task_type=TaskType.SEQ_2_SEQ_LM
)

In [22]:
# TODO: Add LoRA to the LLM model to be trained
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
lora_model = get_peft_model(original_model, lora_config)

In [25]:
# TODO: Print number of parameters, compare LoRA to the original model
print('original model')
print(print_trainable_model_params(model))

print()

print('lora model')
print(print_trainable_model_params(lora_model))

original model
Trainable parameters: 247577856
Total parameters: 247577856
Percentable of trainable parameters: 100.00%

lora model
Trainable parameters: 1769472
Total parameters: 249347328
Percentable of trainable parameters: 0.71%


In [26]:
print(249347328 - 247577856)

1769472


In [29]:
# TODO: Train model with LoRA
output_dir = 'lora-model-summary'

lora_training_args = TrainingArguments(
   output_dir=output_dir,
   learning_rate=1e-3,
   num_train_epochs=1,
)

In [ ]:
# TODO: Create trainer and train model
lora_trainer = Trainer(
   model=lora_model,
   args = lora_training_args,
   train_dataset=tokenized_ds['train'],
   eval_dataset=tokenized_ds['validation']
)

# Run the trainer
#lora_trainer.train()

# Save model
#my_lora_model = 'mymodel'
#lora_trainer.model.save_pretrained(my_lora_model)
#tokenizer.save_pretrained(my_lora_model)

### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [31]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'



In [33]:
# original model
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# load lora model
lora_model = PeftModel.from_pretrained(
   base_model, 
   peft_dialogue_summary_checkpoint,
   torch_dtype=torch.bfloat16,
   is_trainable=False
)

# Load the original model 
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

## Evaluate LoRA model

In [ ]:
# TODO: Evaluate LoRA model against the original 



In [34]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = ds['test'][i]['dialogue']
   s = ds['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

i = 0
i = 1
i = 2
i = 3
i = 4


,label,original_model_summary,lora_model_summary
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...


### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [35]:
# TODO: create ROUGE metrics
rouge = evaluate.load('rouge')

In [37]:
# TODO: Evaluate the original model's result
orig_model_results = rouge.compute(
   references=summaries,
   predictions=orig_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)

lora_model_results = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)

print("Original model results")
print(orig_model_results)

print()

print("LoRA model results")
print(lora_model_results)

Original model results
{'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.13364468864468865), 'rougeLsum': np.float64(0.1336263736263736)}

LoRA model results
{'rouge1': np.float64(0.33973214826086584), 'rouge2': np.float64(0.10249242018904939), 'rougeL': np.float64(0.27287054409005623), 'rougeLsum': np.float64(0.27097500453912726)}


In [ ]:
# TODO: Evaluate with Bleu metrics
